In [1]:
###packages
import numpy as np
import mne
import os
mne.set_log_level('WARNING')

# Spectral analysis

In [2]:
###Will use that later a lot
nyquist_krk = 128
nyquist_wwa = 125

In [4]:
###RUN JUST ONCE 
###creating frequency vectors for band masking
##krk
epochs_krk =  mne.io.read_epochs_eeglab(r"D:\mgr_disc\data_after_EEGlab_improved\Krk\101_psychodel_rest_epoched.set")
freqs_krk = epochs_krk.compute_psd(fmax=nyquist_krk).freqs
np.save(r"D:\mgr_disc\freq_vectors\freqs_krk", freqs_krk)

##wwa
epochs_wwa =  mne.io.read_epochs_eeglab(r"D:\mgr_disc\data_after_EEGlab_improved\Wwa\002_psychodel_rest_epoched.set")
freqs_wwa = epochs_wwa.compute_psd(fmax=nyquist_wwa).freqs
np.save(r"D:\mgr_disc\freq_vectors\freqs_wwa", freqs_wwa)

### *!!!!!*
Remove certain participants

In [34]:
to_get_rid_of = ['020', '024', '025', '028', '082', '083', '117']

os.chdir(r"D:\mgr_disc\Final")
cities = ['Krk', 'Wwa']

for city in cities:
    
    os.chdir(city)
    file_list = os.listdir()

    for file in file_list:
        
        if any(x in file for x in to_get_rid_of):
            
            os.remove(file)
            print(f'{file} succesfully removed')

    os.chdir('..')

In [3]:
###spectral creating tensors of spectral data for average plot

##paths
os.chdir(r"D:\mgr_disc\Final")
cities = ['Krk', 'Wwa']#we have to do it separetely for each city

##directory of each city
for city in cities:
    os.chdir(city)
    files_all = os.listdir()
    files_set = [x for x in files_all if 'set' in x]
    save_dir = os.path.dirname(os.getcwd()) + rf'\{city}_spectral_tensors'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    ##remove participant with wrong conditions
    if city == 'Wwa':
        files_set.remove('026_psychodel_rest_epoched.set')

    for participant in files_set:
        ##load epoch data
        epochs =  mne.io.read_epochs_eeglab(participant)

        ##separate eyes open and eyes closed
        epo_ec = epochs['S 11']
        epo_eo = epochs['S 10']
        
        ##different cities have different sampling frequencies
        if city == 'Krk':
            psd_ec = epo_ec.compute_psd(fmax=nyquist_krk)
            psd_eo = epo_eo.compute_psd(fmax=nyquist_krk)
        elif city == 'Wwa':
            psd_ec = epo_ec.compute_psd(fmax=nyquist_wwa)
            psd_eo = epo_eo.compute_psd(fmax=nyquist_wwa)

        ##save
        psd_ec.save(os.path.join(save_dir, f"{participant[:3]}_epo_ec-psd.hdf5"), overwrite=True)
        psd_eo.save(os.path.join(save_dir, f"{participant[:3]}_epo_eo-psd.hdf5"), overwrite=True)
    
    ##go to the previous directory to enable next iteration of city loop
    os.chdir('..')

#end :)